In [47]:
import json
import os

import mne
import numpy as np
from tqdm.notebook import tqdm

%matplotlib notebook

In [48]:
class NumpyArrayEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

In [49]:
BANDS = ["delta", "theta", "alpha", "beta", "gamma"]

In [53]:
def get_features(folder, channels_list, size=5):
    for filename in os.listdir(f"preprocessed/{folder}"):
        results_filename = f"{filename[:-8]}_part1.json"
        if results_filename in os.listdir(f"features/{folder}"):
            print(filename, "skip")
            continue

        print(filename)

        resulting_features = {}

        raw = mne.io.read_raw_fif(f"preprocessed/{folder}/{filename}", verbose=False)
        epochs = mne.make_fixed_length_epochs(raw, duration=size, preload=True, verbose=False)

        psd_table = epochs.compute_psd(n_jobs=-1, verbose=False).get_data().mean(axis=2).T
        psd = {
            ch_name: psd_array for ch_name, psd_array in zip(raw.ch_names, psd_table)
        }

        for band in tqdm(BANDS, desc="bands"):
            band_epochs = epochs.copy().pick(
                [f"{channel_name}-{band}" for channel_name in channels_list]
            )

            csd = np.array(
                [
                    mne.time_frequency.csd_multitaper(
                        band_epochs[i:i + 1],
                        n_jobs=-1,
                        verbose="ERROR"
                    ).mean().get_data()
                    for i in range(len(band_epochs))
                ]
            )

            for i in range(len(epochs)):
                for channel_1_i, channel_1 in enumerate(channels_list):
                    for channel_2_i, channel_2 in enumerate(channels_list[channel_1_i + 1:]):
                        name = f"{channel_1}_{channel_2}_{band}"

                        if "coh" not in resulting_features:
                            resulting_features["coh"] = {}
                        if name not in resulting_features["coh"]:
                            resulting_features["coh"][name] = []
                        if "plv" not in resulting_features:
                            resulting_features["plv"] = {}
                        if name not in resulting_features["plv"]:
                            resulting_features["plv"][name] = []

                        curr_csd = csd[
                            max(0, i - 2) : min(len(epochs), i + 3), channel_1_i, channel_2_i
                        ]
                        curr_psd_1_mean = psd[f"{channel_1}-{band}"][
                            max(0, i - 2) : min(len(epochs), i + 3)
                        ].mean()
                        curr_psd_2_mean = psd[f"{channel_1}-{band}"][
                            max(0, i - 2) : min(len(epochs), i + 3)
                        ].mean()

                        resulting_features["coh"][name].append(
                            abs(curr_csd.mean()) /
                            np.sqrt(
                                curr_psd_1_mean * curr_psd_2_mean
                            )
                        )
                        resulting_features["plv"][name].append(
                            abs(
                                (curr_csd / np.abs(curr_csd)).mean()
                            )
                        )

        with open(f"features/{folder}/{results_filename}", mode="w", encoding="utf-8") as file:
            file.write(json.dumps(resulting_features, cls=NumpyArrayEncoder, indent=None))

In [54]:
get_features(
    "isruc-sleep",
    ["F3", "C3", "O1", "F4", "C4", "O2"]
)

s1_10_eeg.fif skip
s1_1_eeg.fif skip
s1_2_eeg.fif skip
s1_3_eeg.fif skip
s1_4_eeg.fif skip
s1_5_eeg.fif skip
s1_6_eeg.fif


bands:   0%|          | 0/5 [00:00<?, ?it/s]

s1_7_eeg.fif


bands:   0%|          | 0/5 [00:00<?, ?it/s]

s1_8_eeg.fif


...

bands:   0%|          | 0/5 [00:00<?, ?it/s]

s3_8_eeg.fif


bands:   0%|          | 0/5 [00:00<?, ?it/s]

s3_9_eeg.fif


bands:   0%|          | 0/5 [00:00<?, ?it/s]

In [58]:
get_features(
    "sleep_edf_database_expanded",
    ["Fpz-Cz", "Pz-Oz"]
)

0_eeg.fif


bands:   0%|          | 0/5 [00:00<?, ?it/s]

10_eeg.fif


bands:   0%|          | 0/5 [00:00<?, ?it/s]

11_eeg.fif


...

bands:   0%|          | 0/5 [00:00<?, ?it/s]

8_eeg.fif


bands:   0%|          | 0/5 [00:00<?, ?it/s]

9_eeg.fif


bands:   0%|          | 0/5 [00:00<?, ?it/s]

In [56]:
get_features(
    "eegmat",
    ["F3", "C3", "O1", "F4", "C4", "O2"]
)

0_eeg.fif


bands:   0%|          | 0/5 [00:00<?, ?it/s]

10_eeg.fif


bands:   0%|          | 0/5 [00:00<?, ?it/s]

11_eeg.fif


...

bands:   0%|          | 0/5 [00:00<?, ?it/s]

8_eeg.fif


bands:   0%|          | 0/5 [00:00<?, ?it/s]

9_eeg.fif


bands:   0%|          | 0/5 [00:00<?, ?it/s]

In [57]:
get_features(
    "spis",
    ["F3", "C3", "O1", "F4", "C4", "O2"]
)

0_eeg.fif


bands:   0%|          | 0/5 [00:00<?, ?it/s]

1_eeg.fif


bands:   0%|          | 0/5 [00:00<?, ?it/s]

2_eeg.fif


...

bands:   0%|          | 0/5 [00:00<?, ?it/s]

8_eeg.fif


bands:   0%|          | 0/5 [00:00<?, ?it/s]

9_eeg.fif


bands:   0%|          | 0/5 [00:00<?, ?it/s]